# 一、爬取目标入口站点

In [1]:
import requests
import re

G_TARGET_URL = 'https://store.steampowered.com/search/?category1=998&filter=topsellers'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'
    
}
response = requests.get(G_TARGET_URL, headers=headers,timeout=10)
content = response.text

## 测试匹配 url 的正则表达式

In [2]:
pattern = re.compile('<a href="([a-zA-z]+://[^\s]*?)".*?data-ds-appid=.*?data-ds-tagids=.*?',re.S)
results = re.findall(pattern, content)
#results

## 爬取总页数并收集每个游戏的 url 链接

- 总页数实时变化
- 爬取的详细链接都放入 GAME_URL 该链表中

In [3]:
page_compile = re.compile('page=(.*?)" onclick=',re.S)
content = response.text
pages = re.findall(page_compile, content)
max_page = pages[2]
print('total pages is :',max_page)
GAME_URL = []
pattern = re.compile('<a href="([a-zA-z]+://[^\s]*?)".*?data-ds-appid',re.S)
for page in range(int(max_page)+1):
    targe_url = G_TARGET_URL+'&page='+str(page)
    try :
        response = requests.get(targe_url, headers=headers)
        content = response.text
        results = re.findall(pattern, content)
        for url in results:
            if len(url)< 60 :
                #print("del wrong url :",url)
                results.remove(url)
                
        GAME_URL.extend(results)
        print(targe_url," is OK"+'\r',end="")
    except:
        print(targe_url," is NOT OK"+'\r',end="")

total pages is : 121


## 查看所有详细链接

In [4]:
GAME_URL

['https://store.steampowered.com/app/730/CounterStrike_Global_Offensive/?snr=1_7_7_topsellers_150_1',
 'https://store.steampowered.com/app/570/Dota_2/?snr=1_7_7_topsellers_150_1',
 'https://store.steampowered.com/app/582010/MONSTER_HUNTER_WORLD/?snr=1_7_7_topsellers_150_1',
 'https://store.steampowered.com/app/477160/Human_Fall_Flat/?snr=1_7_7_topsellers_150_1',
 'https://store.steampowered.com/app/239140/Dying_Light/?snr=1_7_7_topsellers_150_1',
 'https://store.steampowered.com/app/632360/Risk_of_Rain_2/?snr=1_7_7_topsellers_150_1',
 'https://store.steampowered.com/sub/88801/?snr=1_7_7_topsellers_150_1',
 'https://store.steampowered.com/app/883710/RESIDENT_EVIL_2__BIOHAZARD_RE2/?snr=1_7_7_topsellers_150_1',
 'https://store.steampowered.com/app/814380/Sekiro_Shadows_Die_Twice/?snr=1_7_7_topsellers_150_1',
 'https://store.steampowered.com/app/230410/Warframe/?snr=1_7_7_topsellers_150_1',
 'https://store.steampowered.com/app/271590/Grand_Theft_Auto_V/?snr=1_7_7_topsellers_150_1',
 'https

## 所有详细链接保存至 urls.txt

In [5]:
with open("urls.txt",'w+') as f:
    for url in GAME_URL:
        f.write(str(url))
        f.write('\n')

# 二、爬取单个游戏的详细信息

In [6]:
TARGET_URL = 'https://store.steampowered.com/app/730/CounterStrike_Global_Offensive/'
response = requests.get(TARGET_URL, headers=headers)
content = response.text


## 获取游戏名

In [7]:
name_pattern = re.compile('<div class="apphub_AppName">(.*?)</div>',re.S)
results = re.findall(name_pattern, content)
results

['Counter-Strike: Global Offensive']

## 获取游戏公司

In [8]:
developer_pattern = re.compile(r'"developers_list".*?<a href=.*?>(.*?)</a>',re.S)
results = re.findall(developer_pattern, content)
results

['Valve']

## 获取好评等级

In [9]:
description_pattern = re.compile('<span class="game_review_summary positive" itemprop="description">(.*?)</span>',re.S)
results = re.findall(description_pattern, content)
results

['Very Positive']

## 获取评价人数

In [10]:
responsive_pattern = re.compile('<span class="responsive_hidden">.*?\\((.*?)\\).*?</span>',re.S)
results = re.findall(responsive_pattern, content)
results[1]

'3,049,009'

## 获取发行时间

In [11]:
date_pattern = re.compile('<div class="date">(.*?)</div>',re.S)
results = re.findall(date_pattern, content)
results

['21 Aug, 2012']

## 获取价格

In [12]:
price_pattern = re.compile('<div class="game_purchase_price price".*?>(.*?)</div>',re.S)
price = re.findall(price_pattern, content)
if len(price) <1:
    bs = etree.HTML(content) 
    price = bs.xpath('//div [@class="discount_prices"]/div[2]')[0].text
price[0].strip()

'Free to Play'

## 获取系统需求

In [13]:
from lxml import etree 
bs = etree.HTML(content) 
content_active = bs.xpath('//div [@class = "game_area_sys_req sysreq_content active"]/div[1]/ul/ul/li')
results= []
for i in content_active:
    results.append(i.xpath('./text()'))
results

[[' Windows® 7/Vista/XP'],
 [' Intel® Core™ 2 Duo E6600 or AMD Phenom™ X3 8750 processor or better'],
 [' 2 GB RAM'],
 [' Video card must be 256 MB or more and should be a DirectX 9-compatible with support for Pixel Shader 3.0'],
 [' Version 9.0c'],
 [' 15 GB available space']]

## 获取游戏标签

In [14]:
table_pattern = re.compile('<a href="https://store.steampowered.com/tags/.*?" class="app_tag" style=.*?>(.*?)</a>',re.S)
results = re.findall(table_pattern, content)
new_res = []
for r in results:
    new_res.append(r.strip())
new_res

['FPS',
 'Multiplayer',
 'Shooter',
 'Action',
 'Team-Based',
 'Competitive',
 'Tactical',
 'First-Person',
 'e-sports',
 'PvP',
 'Online Co-Op',
 'Military',
 'Co-op',
 'Strategy',
 'War',
 'Trading',
 'Difficult',
 'Realistic',
 'Fast-Paced',
 'Moddable']

## 获取语言种类

In [15]:
language_pattern = re.compile('<td style=.*? class="ellipsis">(.*?)</td>',re.S)
results = re.findall(language_pattern, content)
new_res = []
for r in results:
    new_res.append(r.strip())
new_res 

['English',
 'Czech',
 'Danish',
 'Dutch',
 'Finnish',
 'French',
 'German',
 'Hungarian',
 'Italian',
 'Japanese',
 'Korean',
 'Norwegian',
 'Polish',
 'Portuguese',
 'Portuguese - Brazil',
 'Romanian',
 'Russian',
 'Simplified Chinese',
 'Spanish - Spain',
 'Swedish',
 'Thai',
 'Traditional Chinese',
 'Turkish',
 'Bulgarian',
 'Ukrainian']

# 三、爬取所有游戏的详细信息并存成 csv 文件 

- 2000 多页大约耗时 25 分钟

In [18]:
NAME = []
COMPANY = []
PRAISE_LEVEL=[]
COMMENT_NUMBER = []
LANGUAGE = []
PRICE = []
ISSUE_TIME = []
SYSTEM_REQ = []
GGME_TAG = []
index = 0
session = requests.Session()
for url in GAME_URL:
#     index = index+1  #调试 10 页
#     if index>10:
#         break
    session.cookies['cookie'] = 'browserid=1377447744150170549; _ga=GA1.2.1411234599.1555375814; timezoneOffset=28800,0; lastagecheckage=1-0-1993; _gid=GA1.2.1203416908.1556375999; recentapps=%7B%22730%22%3A1556377343%2C%22495140%22%3A1556375909%2C%22456670%22%3A1555400840%2C%22814380%22%3A1555396214%2C%22570%22%3A1555396135%2C%22916440%22%3A1555395961%2C%22567640%22%3A1555395952%2C%22578080%22%3A1555377266%2C%22742230%22%3A1555375831%7D; steamCountry=CN%7Cec11948ab38364bf07ea8d60ab04b516; sessionid=50eaa7ab5eb995868896ba81; birthtime=723139201'

    try:
        response = session.get(url, headers=headers,timeout=10)
        content = response.text
        #游戏名
        name_pattern = re.compile('<div class="apphub_AppName">(.*?)</div>',re.S)
        name = re.findall(name_pattern, content)
        print("name is OK !")
        #公司
        developer_pattern = re.compile(r'"developers_list".*?<a href=.*?>(.*?)</a>',re.S)
        developer = re.findall(developer_pattern, content) 
        print("developer is OK !")
        #评论人数
        responsive_pattern = re.compile('<span class="responsive_hidden">.*?\\((.*?)\\).*?</span>',re.S)
        comment_num = re.findall(responsive_pattern, content)
        print("comment_num is OK !")
        #好评等级
        description_pattern = re.compile('<span class="game_review_summary positive" itemprop="description">(.*?)</span>',re.S)
        description = re.findall(description_pattern, content)
        print("description is OK !")
        #语言种类
        language_pattern = re.compile('<td style=.*? class="ellipsis">(.*?)</td>',re.S)
        languages = re.findall(language_pattern, content)
        language_res = []
        for r in languages:
            language_res.append(r.strip())
        print("languages is OK !")
        #价格
        price_pattern = re.compile('<div class="game_purchase_price price".*?>(.*?)</div>',re.S)
        price = re.findall(price_pattern, content)
        #打折情况
        if len(price) <1: 
            bs = etree.HTML(content) 
            price = bs.xpath('//div [@class="discount_prices"]/div[2]')[0].text
        print("price is OK !")
        #发行日期
        date_pattern = re.compile('<div class="date">(.*?)</div>',re.S)
        date = re.findall(date_pattern, content)
        print("date is OK !")
        #配置要求
        bs = etree.HTML(content) 
        content_active = bs.xpath('//div [@class = "game_area_sys_req sysreq_content active"]/div[1]/ul/ul/li')
        req= []
        for i in content_active:
            req.append(i.xpath('./text()'))
        print("req is OK !")
        #游戏标签
        tag_pattern = re.compile('<a href="https://store.steampowered.com/tags/.*?" class="app_tag" style=.*?>(.*?)</a>',re.S)
        tags = re.findall(tag_pattern, content)
        tags_res = []
        for r in tags:
            tags_res.append(r.strip())     
        print("tags is OK !")
        
        NAME.append(name)
        COMPANY.append(developer)
        PRAISE_LEVEL.append(description)
        if len(comment_num)>=2:
            COMMENT_NUMBER.append(comment_num[1])
        else:
            COMMENT_NUMBER.append('No user reviews')
        LANGUAGE.append(language_res)
        PRICE.append(price[0].strip())
        ISSUE_TIME.append(date)
        SYSTEM_REQ.append(req)
        GGME_TAG.append(tags_res)
        print(url," is OK !")
        
    except:
        #有些网页需要登录账号查看，所以会导致获取数据失败
        print(url," is Not OK !!!!!!!!!!!!!!!!!!!!!!!!!!")

name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/730/CounterStrike_Global_Offensive/?snr=1_7_7_topsellers_150_1  is OK !
name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/570/Dota_2/?snr=1_7_7_topsellers_150_1  is OK !
name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/582010/MONSTER_HUNTER_WORLD/?snr=1_7_7_topsellers_150_1  is OK !
name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/477160/Human_Fall_Flat/?snr=1_7_7_topsellers_150_1  is OK !
name is OK !
developer is OK !
comment_num is OK !


name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/359550/Tom_Clancys_Rainbow_Six_Siege/?snr=1_7_7_topsellers_150_1  is OK !
name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/sub/281610/?snr=1_7_7_topsellers_150_1  is OK !
name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/552990/World_of_Warships/?snr=1_7_7_topsellers_150_1  is OK !
name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/646910/The_Crew_2/?snr=1_7_7_topsellers_150_1  is OK !
name is OK !
developer is OK !
comment_num is OK !
descriptio

name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/958260/DEAD_OR_ALIVE_Xtreme_Venus_Vacation/?snr=1_7_7_topsellers_150_2  is OK !
name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/413150/Stardew_Valley/?snr=1_7_7_topsellers_150_2  is OK !
name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/220440/DmC_Devil_May_Cry/?snr=1_7_7_topsellers_150_3  is OK !
name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/251060/Wargame_Red_Dragon/?snr=1_7_7_topsellers_150_3  is OK !
name is OK !
developer is OK !
c

name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/838350/_The_Scroll_Of_Taiwu/?snr=1_7_7_topsellers_150_4  is OK !
name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/281990/Stellaris/?snr=1_7_7_topsellers_150_4  is OK !
name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/994280/Gujian3/?snr=1_7_7_topsellers_150_4  is OK !
name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/457140/Oxygen_Not_Included/?snr=1_7_7_topsellers_150_4  is OK !
name is OK !
developer is OK !
comment_num is OK !
descriptio

name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/613100/House_Flipper/?snr=1_7_7_topsellers_150_5  is OK !
name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/848450/Subnautica_Below_Zero/?snr=1_7_7_topsellers_150_5  is OK !
name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/838380/DEAD_OR_ALIVE_6/?snr=1_7_7_topsellers_150_5  is OK !
name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/588650/Dead_Cells/?snr=1_7_7_topsellers_150_5  is OK !
name is OK !
developer is OK !
comment_num is OK !
descri

...

name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/491000/Gravity_Cat/?snr=1_7_7_topsellers_150_115  is OK !
name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/493580/Hidden_Object__12_in_1_bundle/?snr=1_7_7_topsellers_150_115  is OK !
name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/864420/Waterside_Chirping/?snr=1_7_7_topsellers_150_115  is OK !
name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/341540/TRIP_Steam_Edition/?snr=1_7_7_topsellers_150_115  is OK !
name is OK !
developer is OK !
c

name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/320320/Darkstone/?snr=1_7_7_topsellers_150_117  is OK !
name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/857060/CCCP_CALLS/?snr=1_7_7_topsellers_150_117  is OK !
name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/429470/Space_Pilgrim_Episode_I_Alpha_Centauri/?snr=1_7_7_topsellers_150_117  is OK !
name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/1056010/_/?snr=1_7_7_topsellers_150_117  is OK !
name is OK !
developer is OK !
comment_num is OK 

name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/810670/NALOGI/?snr=1_7_7_topsellers_150_118  is OK !
name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/981550/GACHIMUCHI_MNLY_PUZZLE/?snr=1_7_7_topsellers_150_118  is OK !
name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/899540/PROTEST/?snr=1_7_7_topsellers_150_118  is OK !
name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/487700/Conan_the_mighty_pig/?snr=1_7_7_topsellers_150_118  is OK !
name is OK !
developer is OK !
comment_num is OK !
de

name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/708450/Pembrey/?snr=1_7_7_topsellers_150_120  is OK !
name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/733090/Mustache_Politics_Shooter/?snr=1_7_7_topsellers_150_120  is OK !
name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/1007210/Trouble_Travel_TT/?snr=1_7_7_topsellers_150_120  is OK !
name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/app/380140/The_Secret_of_Tremendous_Corporation/?snr=1_7_7_topsellers_150_120  is OK !
name is OK !
developer

name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/sub/41008/?snr=1_7_7_topsellers_150_121  is OK !
name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/sub/44166/?snr=1_7_7_topsellers_150_121  is OK !
name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/sub/31292/?snr=1_7_7_topsellers_150_121  is OK !
name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !
req is OK !
tags is OK !
https://store.steampowered.com/sub/69070/?snr=1_7_7_topsellers_150_121  is OK !
name is OK !
developer is OK !
comment_num is OK !
description is OK !
languages is OK !
price is OK !
date is OK !


In [ ]:
import pandas as pd
from pandas.core.frame import DataFrame
 
all_data={"name" : NAME,
   "company" : COMPANY,
   "praise level" : PRAISE_LEVEL,
   "comment number" : COMMENT_NUMBER,
   "language" : LANGUAGE,
   "price" : PRICE,
   "issue time" : ISSUE_TIME,
   "system require" : SYSTEM_REQ,
   "game tags" : GGME_TAG}#将列表转换成字典
data=DataFrame(all_data)#将字典转换成为数据框

In [ ]:
# 简单清洗数据
data_copy = data
def dataFilter(category):
    s = str(category)
    s = s.replace('[','')
    s = s.replace(']','')
    s = s.replace('\'','')
    s = s.replace('¥','')
    return s

data['Name'] = data_copy['name'].apply(dataFilter)
data['Company'] = data_copy['company'].apply(dataFilter)
data['Praise Level'] = data_copy['praise level'].apply(dataFilter)
data['Comment Number'] = data_copy['comment number'].apply(dataFilter)
data['Language'] = data_copy['language'].apply(dataFilter)
data['Price'] = data_copy['price'].apply(dataFilter)
data['Issue time'] = data_copy['issue time'].apply(dataFilter)
data['System Require'] = data_copy['system require'].apply(dataFilter)
data['Game Tags'] = data_copy['game tags'].apply(dataFilter)

del data['name']
del data['company']
del data['praise level']
del data['comment number']
del data['language']
del data['price']
del data['issue time']
del data['system require']
del data['game tags']

In [ ]:
# encoding 防止中文乱码
data.to_csv('all_data.csv',encoding='utf_8_sig')